In [1]:
import pandas as pd
import time
import numpy as np
import spotipy
import math
from spotipy.oauth2 import SpotifyClientCredentials #To access authorised Spotify data
#Require sign-in to Spotify for developers and create an app to obtain client id and client secret.
client_id="######"
client_secret="######"
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=None, retries=10, \
                     status_retries = 10) #spotify object to access API

In [2]:
n=1000
top1000 = pd.read_csv('Spotify streaming artists.csv')
top1000artists = list(top1000['Artist Name'][0:n])

In [3]:
def extract_artist_id(item):
    for counter in list(range(0,2000,50)):
        result=sp.search(item, limit=50, offset= counter, type='artist') # Allow more than 50 searched results for an artist
        #Extract Artist's uri
        artist_result = result['artists']['items']
        for n in range(len(artist_result)):
            if artist_result[n]['name']==item:
                artist_uri = artist_result[n]['uri']
                artist_name = artist_result[n]['name']
                return artist_uri, artist_name
            n+=1

In [4]:
def extract_artists(artist):
    columns = ['artist_name','follower_count','genre','artist_popularity']
    dic_df = {keys: [] for keys in columns} #Creates dictionary for all artists

    for item in artist:
        
        artist_uri, artist_name = extract_artist_id(item)
        results = sp.artist(artist_uri) #Obtain details of a particular artist
        
        #Append information of particular artist to dictionary
        dic_df['artist_name'].append(artist_name)
        dic_df['follower_count'].append(results['followers']['total'])
        dic_df['genre'].append(results['genres'])
        dic_df['artist_popularity'].append(results['popularity'])
 
    summary = pd.DataFrame(dic_df)
    summary.to_csv("Top 1000 artists on Spotify.csv", index=False)
    return summary

In [5]:
extract_artists(list(top1000['Artist Name']))

,artist_name,follower_count,genre,artist_popularity
0,Drake,49816166,"[canadian hip hop, canadian pop, hip hop, pop ...",100
1,Ed Sheeran,70179992,"[pop, uk pop]",93
2,Post Malone,28434185,"[dfw rap, melodic rap, rap]",95
3,Eminem,38231429,"[detroit hip hop, hip hop, rap]",94
4,Ariana Grande,50997684,"[dance pop, pop, post-teen pop]",94
...,...,...,...,...
995,Dadju,2650505,"[francoton, french hip hop, pop urbaine]",78
996,Kina,503623,"[pop, sad rap]",78
997,Vintage Culture,1351010,[brazilian edm],79
998,Theory of a Deadman,1387198,"[alternative metal, canadian metal, canadian r...",71


In [6]:
def albumSongs(uri, name):
    album = uri #assign album uri to a name
    
    #Create keys-values of empty lists inside nested dictionary for album
    columns = ['album','release_date','track_number','track_name','uri','main_artist_name', 'album_popularity']
    
    spotify_albums[album] = {keys: [] for keys in columns} #Creates dictionary for that specific album
    
    result = []
    tracks = sp.album_tracks(album, limit=50) #pull data on album tracks
    result.extend(tracks['items'])
    while tracks['next']:
        tracks = sp.next(tracks) #extend results from data extraction of album tracks that exceed the limit
        time.sleep(0.2)
        result.extend(tracks['items'])
    
    album_result = sp.album(album) #pull data on specific album (popularity)
    time.sleep(0.2)
    
    for n in range(len(result)): #for each song track
        spotify_albums[album]['album'].append(album_result['name'])
        spotify_albums[album]['release_date'].append(album_result['release_date'])
        spotify_albums[album]['track_number'].append(result[n]['track_number'])
        spotify_albums[album]['track_name'].append(result[n]['name'])
        spotify_albums[album]['uri'].append(result[n]['uri'])
        spotify_albums[album]['main_artist_name'].append(name)
        spotify_albums[album]['album_popularity'].append(album_result['popularity'])

In [7]:
def audio_features(uri):
    album = uri #assign album uri to a name
    #Add new key-values to store audio features
    columns = ['duration_ms','key','mode','time_signature','acousticness','danceability','energy','instrumentalness','liveness',
               'loudness','speechiness','tempo','valence','track_popularity','explicit','disc_number']
    spotify_albums[album].update({keys: [] for keys in columns})
    
    #create a track counter
    track_count = 0
    features=[]
    track_total = len(spotify_albums[album]['uri'])
    
    # Extend features list given request can contain maximum of 100 track IDs at one time
    for batch_count in list(range(math.ceil(track_total/100))):
        if track_total>100: 
            features = features + sp.audio_features(spotify_albums[album]['uri'][100*batch_count:100*(batch_count+1)])
            time.sleep(0.2)
            track_total-=100
        else:
            features = features + sp.audio_features(spotify_albums[album]['uri'][100*batch_count:track_total])
            time.sleep(0.2)
            
    for track in spotify_albums[album]['uri']:
        for item in columns[:-3]:
            try:
                spotify_albums[album][item].append(features[track_count][item]) #Append to relevant key-value
            except:
                spotify_albums[album][item].append(None) #Append null value if not found
        
        #popularity, explicit and disc number is stored on a different request
        pop=sp.track(track)
        spotify_albums[album]['track_popularity'].append(pop['popularity'])
        spotify_albums[album]['explicit'].append(pop['explicit'])
        spotify_albums[album]['disc_number'].append(pop['disc_number'])
        track_count+=1

In [8]:
def extract_data(data, start, end):
    columns = ['album','main_artist_name','release_date','album_popularity','track_number','track_name','disc_number','uri',
               'duration_ms','key','mode','time_signature','acousticness','danceability','energy','instrumentalness','liveness',
               'loudness','speechiness','tempo','valence','track_popularity','explicit']
    dic_df = {keys: [] for keys in columns} #Creates dictionary for all albums

    for item in data:
        display(item)
        artist_uri, artist_name = extract_artist_id(item)

        #Pull all of the artist's albums
        albums = []
        results = sp.artist_albums(artist_uri, album_type='album')
        time.sleep(0.2)
        albums.extend(results['items'])
        while results['next']:
            results = sp.next(results)
            time.sleep(0.2)
            albums.extend(results['items'])

        #Store artist's albums' names and uris in separate lists
        album_names = []
        album_uris = []
        for i in range(len(albums)):
            if albums[i]['name'] not in album_names: #Ensure that there are no duplicated albums for each individual artist
                album_names.append(albums[i]['name'])
                album_uris.append(albums[i]['uri'])
        
        global spotify_albums
        spotify_albums = {}
        album_count = 0
        start_time = time.time()
        
        for i in album_uris: #extract songs for each album
            albumSongs(i, artist_name)
            time.sleep(0.2)
            print("Album " + str(album_names[album_count]) + " songs has been added to spotify_albums dictionary")
            album_count+=1 #Updates album count once all tracks have been added
            audio_features(i)
            time.sleep(0.2)
            if album_count % 5 == 0:
                print(str(album_count) + " playlists completed")
                print('Loop #: {}'.format(album_count))
                print('Elapsed Time: {} seconds'.format(time.time() - start_time))

        for album in spotify_albums: #append album and audio features to dictionary
            for feature in spotify_albums[album]:
                dic_df[feature].extend(spotify_albums[album][feature])
        
    # Convert dictionary of track features to data frame
    summary = pd.DataFrame(dic_df) 
    
    # Convert data frame to csv file
    summary.to_csv('Albums for Top 1000 artists ({} to {}).csv'.format(start, end), index=False)
    
    return summary

In [9]:
extract_data(top1000artists[0:100], '1','100') #First 100 artists

'Drake'

Album Dark Lane Demo Tapes songs has been added to spotify_albums dictionary
Album Care Package songs has been added to spotify_albums dictionary
Album So Far Gone songs has been added to spotify_albums dictionary
Album Scorpion songs has been added to spotify_albums dictionary
Album More Life songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 12.990198850631714 seconds
Album Views songs has been added to spotify_albums dictionary
Album What A Time To Be Alive songs has been added to spotify_albums dictionary
Album If You're Reading This It's Too Late songs has been added to spotify_albums dictionary
Album Nothing Was The Same (Deluxe) songs has been added to spotify_albums dictionary
Album Nothing Was The Same songs has been added to spotify_albums dictionary
10 playlists completed
Loop #: 10
Elapsed Time: 24.498607397079468 seconds
Album Take Care songs has been added to spotify_albums dictionary
Album Take Care (Deluxe) songs has been add

'Ed Sheeran'

Album No.6 Collaborations Project songs has been added to spotify_albums dictionary
Album ÷ (Deluxe) songs has been added to spotify_albums dictionary
Album x (Wembley Edition) songs has been added to spotify_albums dictionary
Album 5 songs has been added to spotify_albums dictionary
Album x (Deluxe Edition) songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 13.86432671546936 seconds
Album Live at the Bedford songs has been added to spotify_albums dictionary
Album Loose Change songs has been added to spotify_albums dictionary
Album + songs has been added to spotify_albums dictionary
Album No.5 Collaborations Project songs has been added to spotify_albums dictionary


'Post Malone'

...

'Luis Fonsi'

Album VIDA songs has been added to spotify_albums dictionary
Album 2En1 songs has been added to spotify_albums dictionary
Album Lo Mejor De... songs has been added to spotify_albums dictionary
Album 8 songs has been added to spotify_albums dictionary
Album 8 (Deluxe) songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 13.018311738967896 seconds
Album Muve Sessions: 8 songs has been added to spotify_albums dictionary
Album Romances songs has been added to spotify_albums dictionary
Album Tierra Firme songs has been added to spotify_albums dictionary
Album Palabras Del Silencio songs has been added to spotify_albums dictionary
Album Palabras Del Silencio (Edición Aniversario - Un Año Despues) songs has been added to spotify_albums dictionary
10 playlists completed
Loop #: 10
Elapsed Time: 24.489007472991943 seconds
Album Éxitos 98:06 songs has been added to spotify_albums dictionary
Album Exitos: 98:06 songs has been added to spotify_albums dict

'Jonas Blue'

Album Blue songs has been added to spotify_albums dictionary


,album,main_artist_name,release_date,album_popularity,track_number,track_name,disc_number,uri,duration_ms,key,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,track_popularity,explicit
0,Dark Lane Demo Tapes,Drake,2020-05-01,88,1,Deep Pockets,1,spotify:track:3IvMYBE7A3c7to1aEcfFJk,222929.0,7.0,...,0.473,0.824,0.000000,0.6050,-3.680,0.1630,77.888,0.3740,65,True
1,Dark Lane Demo Tapes,Drake,2020-05-01,88,2,When To Say When,1,spotify:track:5TCBWmEBrin7etRa4Lswr1,223124.0,1.0,...,0.410,0.820,0.000000,0.5380,-6.808,0.5330,170.718,0.5260,66,True
2,Dark Lane Demo Tapes,Drake,2020-05-01,88,3,Chicago Freestyle (feat. Giveon),1,spotify:track:4wVOKKEHUJxHCFFNUWDn0B,220488.0,10.0,...,0.735,0.449,0.000000,0.1130,-7.507,0.3470,122.947,0.0397,85,True
3,Dark Lane Demo Tapes,Drake,2020-05-01,88,4,Not You Too (feat. Chris Brown),1,spotify:track:3Q4gttWQ6hxqWOa3tHoTNi,269680.0,9.0,...,0.458,0.452,0.000019,0.0703,-9.299,0.0470,86.318,0.3160,70,True
4,Dark Lane Demo Tapes,Drake,2020-05-01,88,5,Toosie Slide,1,spotify:track:466cKvZn1j45IpxDdYZqdA,247059.0,1.0,...,0.830,0.490,0.000003,0.1130,-8.820,0.2090,81.604,0.8450,82,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20246,Blue,Jonas Blue,2018-11-09,81,11,I See Love,1,spotify:track:0Ffmo50Ce6QpMDcHyFCtKL,173220.0,2.0,...,0.736,0.703,0.000000,0.3250,-3.899,0.0440,123.047,0.7540,50,False
20247,Blue,Jonas Blue,2018-11-09,81,12,Drink To You,1,spotify:track:3FCCxdjUyPVLUrpCL2VUwC,210120.0,1.0,...,0.742,0.813,0.000000,0.0607,-5.103,0.0574,120.010,0.7020,43,False
20248,Blue,Jonas Blue,2018-11-09,81,13,Come Through,1,spotify:track:2so1nPxGyQRG9pUH1d09zS,178200.0,0.0,...,0.435,0.746,0.000000,0.0476,-4.880,0.0759,105.043,0.3650,39,False
20249,Blue,Jonas Blue,2018-11-09,81,14,Supernova,1,spotify:track:2WiyIbpBw4OoxqA9AEUHfZ,187307.0,9.0,...,0.595,0.656,0.000000,0.1370,-7.195,0.0634,139.982,0.5040,37,False


In [10]:
extract_data(top1000artists[100:200], '101','200') #Next 100 artists

'Bebe Rexha'

Album Expectations songs has been added to spotify_albums dictionary


'Bob Marley & The Wailers'

Album Live At The Rainbow, 4th June 1977 (Remastered 2020) songs has been added to spotify_albums dictionary
Album Exodus 40 songs has been added to spotify_albums dictionary
Album Exodus 40 (40th Anniversary Deluxe Edition) songs has been added to spotify_albums dictionary
Album Easy Skanking In Boston '78 songs has been added to spotify_albums dictionary
Album Marley (The Original Soundtrack) songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 12.65120267868042 seconds
Album Live Forever: The Stanley Theatre, Pittsburgh, PA, 9/23/1980 songs has been added to spotify_albums dictionary
Album Live Forever: The Stanley Theatre, Pittsburgh, PA, September 23, 1980 (Highlights) songs has been added to spotify_albums dictionary
Album B Is For Bob songs has been added to spotify_albums dictionary
Album Bob Marley & The Waillers Springtime Anthology - meets Lee ''Scratch'' Perry Sessions - songs has been added to spotify_albums dictionary
Album Exodu

'James Arthur'

...

'Kid Cudi'

Album KIDS SEE GHOSTS songs has been added to spotify_albums dictionary
Album Passion, Pain & Demon Slayin' songs has been added to spotify_albums dictionary
Album Speedin' Bullet 2 Heaven songs has been added to spotify_albums dictionary
Album KiD CuDi presents SATELLITE FLIGHT: The journey to Mother Moon songs has been added to spotify_albums dictionary
Album Indicud songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 11.841700077056885 seconds
Album Man On The Moon II: The Legend Of Mr. Rager songs has been added to spotify_albums dictionary
Album Man On The Moon II: The Legend Of Mr. Rager (Explicit Version) songs has been added to spotify_albums dictionary
Album Man On The Moon II: The Legend Of Mr. Rager (Edited Version) songs has been added to spotify_albums dictionary
Album Man On The Moon: The End Of Day songs has been added to spotify_albums dictionary
Album Day N Nite songs has been added to spotify_albums dictionary
10 playlists c

'Tove Lo'

Album Sunshine Kitty (Paw Prints Edition) songs has been added to spotify_albums dictionary
Album Sunshine Kitty songs has been added to spotify_albums dictionary
Album BLUE LIPS (lady wood phase II) songs has been added to spotify_albums dictionary
Album Lady Wood songs has been added to spotify_albums dictionary
Album Divergente 3 : Au delà du mur (Bande son originale du film) songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 11.609718561172485 seconds
Album Queen Of The Clouds (Blueprint Edition) songs has been added to spotify_albums dictionary
Album Queen Of The Clouds songs has been added to spotify_albums dictionary


,album,main_artist_name,release_date,album_popularity,track_number,track_name,disc_number,uri,duration_ms,key,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,track_popularity,explicit
0,Expectations,Bebe Rexha,2018-06-22,78,1,Ferrari,1,spotify:track:3hfPmifit8Rtc0zj9WYQlU,212543.0,8.0,...,0.577,0.480,0.000000,0.1040,-5.908,0.0297,76.540,0.4220,53,False
1,Expectations,Bebe Rexha,2018-06-22,78,2,I'm a Mess,1,spotify:track:04ZTP5KsCypmtCmQg5tH9R,195519.0,11.0,...,0.630,0.694,0.000000,0.0719,-6.257,0.0253,97.005,0.2160,78,False
2,Expectations,Bebe Rexha,2018-06-22,78,3,2 Souls on Fire (feat. Quavo),1,spotify:track:2dMUJy0El3BcQUy9gP9gt0,170297.0,4.0,...,0.713,0.592,0.000000,0.3380,-4.621,0.0272,106.013,0.6190,53,False
3,Expectations,Bebe Rexha,2018-06-22,78,4,Shining Star,1,spotify:track:1B3o1yfiEDSQ0G5L9TDbhp,186373.0,11.0,...,0.751,0.550,0.000053,0.1860,-8.425,0.0457,130.092,0.5700,46,True
4,Expectations,Bebe Rexha,2018-06-22,78,5,Knees,1,spotify:track:7IOxauZfx5l18mYfX0GZm6,206581.0,11.0,...,0.777,0.577,0.000001,0.0869,-6.872,0.0357,109.019,0.3810,66,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26786,Queen Of The Clouds,Tove Lo,2015-02-20,36,17,Stay High - Habits Remix,1,spotify:track:2TkiK1Hbiaww3ZjOVX1qqp,258933.0,4.0,...,0.735,0.686,0.871000,0.0933,-7.562,0.0366,121.011,0.0656,28,False
26787,Queen Of The Clouds,Tove Lo,2015-02-20,36,18,Over,1,spotify:track:1r2V4PXsPqjlMtCYqyTXUV,224187.0,0.0,...,0.607,0.610,0.000000,0.0900,-5.628,0.0371,156.988,0.3880,21,True
26788,Queen Of The Clouds,Tove Lo,2015-02-20,36,19,Out Of Mind - Bonus Track,1,spotify:track:0ljGhtL9x23PtnqEjwQK2O,189987.0,6.0,...,0.698,0.772,0.000000,0.6730,-4.713,0.0493,130.071,0.7120,17,False
26789,Queen Of The Clouds,Tove Lo,2015-02-20,36,20,Paradise,1,spotify:track:26GKJ1kP5fzfui03Sv1xxv,183507.0,0.0,...,0.587,0.789,0.000000,0.1890,-4.498,0.0520,159.976,0.4730,19,False


In [11]:
extract_data(top1000artists[200:300], '201','300') #Next 100 artists

'Kings of Leon'

Album WALLS songs has been added to spotify_albums dictionary
Album Mechanical Bull (Expanded Edition) songs has been added to spotify_albums dictionary
Album Come Around Sundown (Expanded Edition) songs has been added to spotify_albums dictionary
Album Only By The Night songs has been added to spotify_albums dictionary
Album Because Of The Times songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 10.150201797485352 seconds
Album Aha Shake Heartbreak songs has been added to spotify_albums dictionary
Album Youth And Young Manhood songs has been added to spotify_albums dictionary


'Paulo Londra'

Album Homerun songs has been added to spotify_albums dictionary


'Luke Bryan'

...

'ABBA'

Album Live At Wembley Arena songs has been added to spotify_albums dictionary
Album The Visitors songs has been added to spotify_albums dictionary
Album Super Trouper songs has been added to spotify_albums dictionary
Album Abba songs has been added to spotify_albums dictionary
Album Thank You For The Music songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 15.386864185333252 seconds
Album Abba Live songs has been added to spotify_albums dictionary
Album The Visitors (Deluxe Edition) songs has been added to spotify_albums dictionary
Album Voulez-Vous songs has been added to spotify_albums dictionary
Album The Album songs has been added to spotify_albums dictionary
Album Arrival songs has been added to spotify_albums dictionary
10 playlists completed
Loop #: 10
Elapsed Time: 25.70776653289795 seconds
Album Waterloo (Deluxe Edition) songs has been added to spotify_albums dictionary
Album Waterloo songs has been added to spotify_albums dictionar

'Jason Aldean'

Album 9 songs has been added to spotify_albums dictionary
Album Rearview Town songs has been added to spotify_albums dictionary
Album They Don't Know songs has been added to spotify_albums dictionary
Album Old Boots, New Dirt songs has been added to spotify_albums dictionary
Album Old Boots, New Dirt (Commentary) songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 11.32241940498352 seconds
Album Night Train songs has been added to spotify_albums dictionary
Album My Kinda Party songs has been added to spotify_albums dictionary
Album Wide Open songs has been added to spotify_albums dictionary
Album Relentless songs has been added to spotify_albums dictionary
Album Jason Aldean songs has been added to spotify_albums dictionary
10 playlists completed
Loop #: 10
Elapsed Time: 21.691579580307007 seconds


,album,main_artist_name,release_date,album_popularity,track_number,track_name,disc_number,uri,duration_ms,key,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,track_popularity,explicit
0,WALLS,Kings of Leon,2016-10-14,70,1,Waste A Moment,1,spotify:track:5LlsD7LdSMkGV4Iu0a2Zq0,183093.0,2.0,...,0.436,0.850,0.740000,0.1280,-4.860,0.0617,152.644,0.537,66,False
1,WALLS,Kings of Leon,2016-10-14,70,2,Reverend,1,spotify:track:5cm47Nc1MWBVIkRFaXxvHY,234080.0,0.0,...,0.448,0.815,0.686000,0.3160,-5.951,0.0361,124.808,0.217,58,False
2,WALLS,Kings of Leon,2016-10-14,70,3,Around The World,1,spotify:track:06ClI3XK2FPR5uP8tPcFS1,214040.0,1.0,...,0.575,0.854,0.676000,0.3510,-5.847,0.0362,113.075,0.587,57,False
3,WALLS,Kings of Leon,2016-10-14,70,4,Find Me,1,spotify:track:2mwnYvlLVP465u9V9wjUGj,245573.0,1.0,...,0.400,0.946,0.853000,0.1550,-3.966,0.0634,161.591,0.611,61,False
4,WALLS,Kings of Leon,2016-10-14,70,5,Over,1,spotify:track:4jyZ3I1hYRcOkI8RJhxgCb,370587.0,2.0,...,0.414,0.799,0.507000,0.1140,-8.253,0.0335,126.852,0.283,55,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18700,Jason Aldean,Jason Aldean,2005-07-26,62,7,I'm Just a Man,1,spotify:track:0SUEomEoBOJIsSar9hmo10,197573.0,5.0,...,0.679,0.748,0.000003,0.1540,-4.025,0.0259,103.066,0.723,36,False
18701,Jason Aldean,Jason Aldean,2005-07-26,62,8,You're the Love I Wanna Be In,1,spotify:track:0YhXneBTwdtFbSVMzqkzmd,197013.0,8.0,...,0.615,0.831,0.000007,0.2540,-2.668,0.0394,118.055,0.555,39,False
18702,Jason Aldean,Jason Aldean,2005-07-26,62,9,Good to Go,1,spotify:track:6wOfzAZOpPMIKJadjy2My5,240600.0,0.0,...,0.560,0.675,0.000000,0.2790,-3.715,0.0342,77.547,0.372,37,False
18703,Jason Aldean,Jason Aldean,2005-07-26,62,10,I Believe in Ghosts,1,spotify:track:6Jf24PIGPBQ1gnTmaPQ5pg,205853.0,5.0,...,0.465,0.751,0.000000,0.0965,-4.045,0.0367,180.084,0.268,33,False


In [12]:
extract_data(top1000artists[300:400], '301','400') #Next 100 artists

'Joji'

Album Nectar songs has been added to spotify_albums dictionary
Album BALLADS 1 songs has been added to spotify_albums dictionary
Album In Tongues (Deluxe) songs has been added to spotify_albums dictionary
Album In Tongues Deluxe songs has been added to spotify_albums dictionary


'Maná'

Album Cama Incendiada songs has been added to spotify_albums dictionary
Album Você É Minha Religião: O Melhor Do Maná songs has been added to spotify_albums dictionary
Album Você É Minha Religião: O Melhor Do Maná (Deluxe Version) songs has been added to spotify_albums dictionary
Album Exiliados En La Bahía: Lo Mejor De Maná (Edición Deluxe) songs has been added to spotify_albums dictionary
Album Drama Y Luz Edición Deluxe songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 12.984429836273193 seconds
Album Drama Y Luz songs has been added to spotify_albums dictionary
Album Drama Y Luz (2020 Remasterizado) songs has been added to spotify_albums dictionary
Album Mana songs has been added to spotify_albums dictionary
Album Arde El Cielo songs has been added to spotify_albums dictionary
Album Amar es Combatir songs has been added to spotify_albums dictionary
10 playlists completed
Loop #: 10
Elapsed Time: 23.586632013320923 seconds
Album Amar Es 

'YNW Melly'

...

'Rich The Kid'

Album BOSS MAN songs has been added to spotify_albums dictionary
Album Rich Forever 4 songs has been added to spotify_albums dictionary
Album The World Is Yours 2 songs has been added to spotify_albums dictionary
Album The World Is Yours songs has been added to spotify_albums dictionary


'Jhené Aiko'

Album Chilombo (Deluxe) songs has been added to spotify_albums dictionary
Album Chilombo songs has been added to spotify_albums dictionary
Album Trip songs has been added to spotify_albums dictionary
Album Souled Out (Deluxe) songs has been added to spotify_albums dictionary
Album Souled Out songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 13.505244255065918 seconds
Album Sail Out songs has been added to spotify_albums dictionary


,album,main_artist_name,release_date,album_popularity,track_number,track_name,disc_number,uri,duration_ms,key,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,track_popularity,explicit
0,Nectar,Joji,2020-09-25,91,1,Ew,1,spotify:track:7xcUWyLh6fIMsYjoVI7NnZ,207710.0,10.0,...,0.480,0.373,0.067000,0.1200,-9.297,0.0278,114.078,0.0376,79,False
1,Nectar,Joji,2020-09-25,91,2,MODUS,1,spotify:track:5SjtAe7xzUgHDtNTmt8zS7,207265.0,7.0,...,0.738,0.533,0.000252,0.1150,-7.617,0.0492,132.008,0.2620,78,False
2,Nectar,Joji,2020-09-25,91,3,Tick Tock,1,spotify:track:5E7p95xh9Ul4vZp4LEoJwi,132215.0,4.0,...,0.781,0.613,0.007130,0.0976,-5.582,0.0524,80.530,0.6130,77,False
3,Nectar,Joji,2020-09-25,91,4,Daylight,1,spotify:track:6Ww8GHdPCl8MqZBhPn4LKd,163906.0,7.0,...,0.526,0.751,0.000112,0.0954,-7.573,0.0479,163.933,0.7390,0,True
4,Nectar,Joji,2020-09-25,91,5,Upgrade,1,spotify:track:5xNNJKrjobta5ksOpMLYd1,89917.0,0.0,...,0.678,0.418,0.031300,0.1370,-11.393,0.0985,159.867,0.5870,75,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25680,Sail Out,Jhené Aiko,2013-01-01,70,3,Stay Ready (What A Life),1,spotify:track:5nkUIVKqOqdpB6ApKgEMkv,382760.0,8.0,...,0.347,0.493,0.003660,0.1270,-11.548,0.2900,82.887,0.3080,62,True
25681,Sail Out,Jhené Aiko,2013-01-01,70,4,WTH,1,spotify:track:1kuNRcIImZO3vGTBAwSN06,199187.0,10.0,...,0.412,0.486,0.000010,0.0947,-7.479,0.0599,89.222,0.4640,51,True
25682,Sail Out,Jhené Aiko,2013-01-01,70,5,The Worst,1,spotify:track:2wBCrtJS3E3TimRZ5MElTI,254493.0,1.0,...,0.539,0.414,0.000001,0.1430,-9.156,0.0601,94.895,0.1230,69,True
25683,Sail Out,Jhené Aiko,2013-01-01,70,6,3:16AM,1,spotify:track:4QoFwqgLw1CtoeSv6N80mU,200720.0,2.0,...,0.602,0.428,0.046900,0.1220,-10.980,0.1290,150.014,0.0464,53,False


In [13]:
extract_data(top1000artists[400:500], '401','500') #Next 100 artists

'The Doors'

Album The Soft Parade (50th Anniversary Deluxe Edition) songs has been added to spotify_albums dictionary
Album Waiting for the Sun (50th Anniversary Deluxe Edition) songs has been added to spotify_albums dictionary
Album Live at the Isle of Wight Festival 1970 songs has been added to spotify_albums dictionary
Album Strange Days (50th Anniversary Expanded Edition) [2017 Remaster] songs has been added to spotify_albums dictionary
Album The Doors (50th Anniversary Deluxe Edition) songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 13.944585800170898 seconds
Album London Fog 1966 (Live) songs has been added to spotify_albums dictionary
Album American Nights - In Concert songs has been added to spotify_albums dictionary
Album The Doors (Original Soundtrack Recording) songs has been added to spotify_albums dictionary
Album An American Prayer songs has been added to spotify_albums dictionary
Album Full Circle songs has been added to spotify_albums 

'Years & Years'

Album Palo Santo (Deluxe) songs has been added to spotify_albums dictionary
Album Communion (Deluxe) songs has been added to spotify_albums dictionary


'Danny Ocean'

...

'Portugal. The Man'

Album Woodstock songs has been added to spotify_albums dictionary
Album Evil Friends songs has been added to spotify_albums dictionary
Album In the Mountain in the Cloud songs has been added to spotify_albums dictionary
Album American Ghetto songs has been added to spotify_albums dictionary
Album The Majestic Majesty songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 9.633011817932129 seconds
Album The Satanic Satanist songs has been added to spotify_albums dictionary
Album Censored Colors songs has been added to spotify_albums dictionary
Album Church Mouth songs has been added to spotify_albums dictionary
Album Waiter: "You Vultures!" songs has been added to spotify_albums dictionary


'Pedro Capó'

Album MUNAY songs has been added to spotify_albums dictionary
Album En Letra de Otro songs has been added to spotify_albums dictionary
Album Aquila songs has been added to spotify_albums dictionary
Album Pedro Capó songs has been added to spotify_albums dictionary
Album Fuego Y Amor songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 9.8101487159729 seconds


,album,main_artist_name,release_date,album_popularity,track_number,track_name,disc_number,uri,duration_ms,key,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,track_popularity,explicit
0,The Soft Parade (50th Anniversary Deluxe Edition),The Doors,2019-11-01,52,1,Tell All The People - 2019 Remaster,1,spotify:track:2Z9jLTcCcHWpiW3WF23l6N,200649.0,4.0,...,0.380,0.334,0.000000,0.2480,-14.763,0.0305,172.803,0.682,39,False
1,The Soft Parade (50th Anniversary Deluxe Edition),The Doors,2019-11-01,52,2,Touch Me - 2019 Remaster,1,spotify:track:7Lwx2edo0NR7KJKmBo1jE5,192213.0,1.0,...,0.476,0.628,0.007100,0.0852,-13.896,0.0558,110.201,0.745,40,False
2,The Soft Parade (50th Anniversary Deluxe Edition),The Doors,2019-11-01,52,3,Shaman's Blues - 2019 Remaster,1,spotify:track:7jBBX9nVQbme8FBKqg2cXb,287925.0,5.0,...,0.510,0.422,0.000088,0.1060,-15.035,0.0293,145.985,0.728,37,False
3,The Soft Parade (50th Anniversary Deluxe Edition),The Doors,2019-11-01,52,4,Do It - 2019 Remaster,1,spotify:track:7J9PRTUCnpydYLujduO9KZ,188959.0,9.0,...,0.379,0.564,0.267000,0.4990,-14.651,0.0353,98.895,0.512,35,False
4,The Soft Parade (50th Anniversary Deluxe Edition),The Doors,2019-11-01,52,5,Easy Ride - 2019 Remaster,1,spotify:track:713zX3J5MuHXj3trcqxWFA,159306.0,0.0,...,0.627,0.747,0.000000,0.1310,-13.668,0.0339,120.579,0.733,35,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22473,Fuego Y Amor,Pedro Capó,2007-12-18,31,7,Ay Amor,1,spotify:track:2ETYpG2vc24Y2CB6I7gvLv,203293.0,9.0,...,0.692,0.780,0.000000,0.2600,-5.174,0.0300,120.087,0.697,15,False
22474,Fuego Y Amor,Pedro Capó,2007-12-18,31,8,Sola,1,spotify:track:1aAvN39MFND4auSfMEyUJG,248267.0,0.0,...,0.548,0.536,0.000000,0.3810,-8.650,0.0549,136.027,0.305,17,False
22475,Fuego Y Amor,Pedro Capó,2007-12-18,31,9,El Precio,1,spotify:track:2XaFc3kEV9PK7cHIoAswvT,219773.0,0.0,...,0.691,0.650,0.000000,0.1190,-6.038,0.0664,95.989,0.906,19,False
22476,Fuego Y Amor,Pedro Capó,2007-12-18,31,10,La Semana,1,spotify:track:3BqsJmnJoJR5ADkTYcYK4V,175200.0,7.0,...,0.614,0.944,0.000000,0.0557,-4.305,0.0823,112.056,0.722,11,False


In [14]:
extract_data(top1000artists[500:600], '501','600') #Next 100 artists

'Joan Sebastian'

Album Atemporal songs has been added to spotify_albums dictionary
Album Mi Mexico Querido songs has been added to spotify_albums dictionary
Album Pegadito Al Corazón songs has been added to spotify_albums dictionary
Album El Último Jaripeo (En Vivo) songs has been added to spotify_albums dictionary
Album Por Siempre Joan Sebastian El Poeta Del Pueblo songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 10.609586715698242 seconds
Album Gracias por Tanto Amor songs has been added to spotify_albums dictionary
Album Antología: El Poeta Del Pueblo, Vol. 1 – Sus Primeros Éxitos songs has been added to spotify_albums dictionary
Album Antología: El Poeta Del Pueblo, Vol. 2 – Más Éxitos songs has been added to spotify_albums dictionary
Album Antología: El Poeta Del Pueblo, Vol. 3 – Sus Grandes Baladas songs has been added to spotify_albums dictionary
Album Antología: El Poeta del Pueblo songs has been added to spotify_albums dictionary
10 playlists com

Album El Peor De Tus Antojos songs has been added to spotify_albums dictionary
Album Joan Sebastian Con Tambora (Vol. 3) songs has been added to spotify_albums dictionary
85 playlists completed
Loop #: 85
Elapsed Time: 169.69659447669983 seconds
Album Joan Sebastian Con Tambora songs has been added to spotify_albums dictionary
Album Tambora, Vol. 2 songs has been added to spotify_albums dictionary
Album Norteño, Vol. 2 songs has been added to spotify_albums dictionary


'WALK THE MOON'

Album What If Nothing songs has been added to spotify_albums dictionary
Album You Are Not Alone (Live At The Greek) songs has been added to spotify_albums dictionary
Album TALKING IS HARD songs has been added to spotify_albums dictionary
Album TALKING IS HARD (Expanded Edition) songs has been added to spotify_albums dictionary
Album Walk The Moon songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 10.778987884521484 seconds
Album Walk The Moon (Expanded Edition) songs has been added to spotify_albums dictionary


...

'M83'

Album DSVII songs has been added to spotify_albums dictionary
Album Knife + Heart (Original Motion Picture Soundtrack) songs has been added to spotify_albums dictionary
Album Knife + Heart (Original Soundtrack) songs has been added to spotify_albums dictionary
Album Knife + Heart (Official Soundtrack) songs has been added to spotify_albums dictionary
Album Go! (Mai Lan Remixes) songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 13.326836585998535 seconds
Album Go! (Remixes) songs has been added to spotify_albums dictionary
Album Junk songs has been added to spotify_albums dictionary
Album Before the Dawn Heals Us (Remixes & B-Sides) songs has been added to spotify_albums dictionary
Album Dead Cities, Red Seas & Lost Ghosts (Remixes & B-Sides) songs has been added to spotify_albums dictionary
Album You And The Night songs has been added to spotify_albums dictionary
10 playlists completed
Loop #: 10
Elapsed Time: 22.46906876564026 seconds
Albu

'Ice Cube'

Album Everythangs Corrupt songs has been added to spotify_albums dictionary
Album Death Certificate (25th Anniversary Edition) songs has been added to spotify_albums dictionary
Album Woodstock '99 (Live) songs has been added to spotify_albums dictionary
Album I Am The West songs has been added to spotify_albums dictionary
Album I Am the West songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 12.038074970245361 seconds
Album Raw Footage songs has been added to spotify_albums dictionary
Album Raw Footage (Edited) songs has been added to spotify_albums dictionary
Album Laugh Now, Cry Later songs has been added to spotify_albums dictionary
Album War & Peace songs has been added to spotify_albums dictionary
Album War & Peace Vol. 1 (The War Disc) songs has been added to spotify_albums dictionary
10 playlists completed
Loop #: 10
Elapsed Time: 24.12352466583252 seconds
Album Lethal Injection songs has been added to spotify_albums dictionary
Album 

,album,main_artist_name,release_date,album_popularity,track_number,track_name,disc_number,uri,duration_ms,key,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,track_popularity,explicit
0,Atemporal,Joan Sebastian,2020-07-13,65,1,Atemporal,1,spotify:track:3NXqYt8NREdk3UwvJbCTB9,96933.0,9.0,...,0.636,0.315,0.000029,0.1110,-12.669,0.0985,91.546,0.358,52,False
1,Atemporal,Joan Sebastian,2020-07-13,65,2,Se Va A Arrepentir,1,spotify:track:3tV7TCBFRB4dZ03gYwZ7Ax,198867.0,9.0,...,0.650,0.597,0.000013,0.0978,-6.427,0.0302,117.069,0.500,59,False
2,Atemporal,Joan Sebastian,2020-07-13,65,3,Trampa,1,spotify:track:0ewlhCDP5dmeC8iLDqJoOk,213693.0,4.0,...,0.628,0.588,0.000000,0.3540,-7.818,0.0344,82.990,0.594,51,False
3,Atemporal,Joan Sebastian,2020-07-13,65,4,Que No Que No,1,spotify:track:0qkzYsQTUzqA7ncl7Vk3g1,214280.0,9.0,...,0.836,0.719,0.000000,0.0626,-7.325,0.0564,92.010,0.964,50,False
4,Atemporal,Joan Sebastian,2020-07-13,65,5,Que Sea,1,spotify:track:3LBnikk8IS3TSBbVUn86eA,169560.0,0.0,...,0.785,0.624,0.000026,0.1340,-6.644,0.0397,97.058,0.899,52,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21922,AmeriKKKa's Most Wanted,Ice Cube,1990-05-16,32,12,Endangered Species (Tales From The Darkside) -...,1,spotify:track:6EBlaVONFXtiiihUatFf7l,250693.0,1.0,...,0.768,0.954,0.000005,0.6960,-5.393,0.1990,107.992,0.474,11,False
21923,AmeriKKKa's Most Wanted,Ice Cube,1990-05-16,32,13,Jackin' For Beats,1,spotify:track:6BhtJJzSnqOZAbvOXxPYoZ,175171.0,7.0,...,0.875,0.931,0.000000,0.8220,-5.732,0.3180,104.150,0.512,18,False
21924,AmeriKKKa's Most Wanted,Ice Cube,1990-05-16,32,14,The Product,1,spotify:track:31QLcCpZSSIUPGH1QLmmjL,215000.0,11.0,...,0.780,0.895,0.000000,0.3250,-5.149,0.2630,110.150,0.686,11,False
21925,AmeriKKKa's Most Wanted,Ice Cube,1990-05-16,32,15,Dead Homiez,1,spotify:track:3sRA65Ty3ZuiXd2mYgB1X1,233173.0,10.0,...,0.716,0.750,0.000002,0.3310,-6.742,0.2600,168.435,0.607,13,False


In [15]:
extract_data(top1000artists[600:700], '601','700') #Next 100 artists

'Snow Patrol'

Album Reworked songs has been added to spotify_albums dictionary
Album Wildness (Deluxe) songs has been added to spotify_albums dictionary
Album Fallen Empires songs has been added to spotify_albums dictionary
Album Fallen Empires (Japanese Edition) songs has been added to spotify_albums dictionary
Album Late Night Tales: Snow Patrol songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 10.680915117263794 seconds
Album Up To Now songs has been added to spotify_albums dictionary
Album A Hundred Million Suns songs has been added to spotify_albums dictionary
Album Final Straw songs has been added to spotify_albums dictionary
Album Eyes Open (Eastern European Version) songs has been added to spotify_albums dictionary
Album Eyes Open songs has been added to spotify_albums dictionary
10 playlists completed
Loop #: 10
Elapsed Time: 22.044854640960693 seconds
Album Snow Patrol: Sessions@AOL songs has been added to spotify_albums dictionary
Album When I

'The Clash'

Album Live At Shea Stadium songs has been added to spotify_albums dictionary
Album Live at Shea Stadium (Remastered) songs has been added to spotify_albums dictionary
Album From Here To Eternity Live songs has been added to spotify_albums dictionary
Album From Here to Eternity (Live) [Remastered] songs has been added to spotify_albums dictionary
Album Raw and Unfiltered songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 11.663802146911621 seconds
Album Cut The Crap songs has been added to spotify_albums dictionary
Album Combat Rock songs has been added to spotify_albums dictionary
Album Combat Rock (Remastered) songs has been added to spotify_albums dictionary
Album Sandinista! (Remastered) songs has been added to spotify_albums dictionary
Album London Calling (Expanded Edition) songs has been added to spotify_albums dictionary
10 playlists completed
Loop #: 10
Elapsed Time: 25.88427495956421 seconds
Album London Calling (Remastered) songs h

'fun.'

...

'Nipsey Hussle'

Album Nipsey Hussle & YG songs has been added to spotify_albums dictionary
Album Victory Lap songs has been added to spotify_albums dictionary
Album No Pressure songs has been added to spotify_albums dictionary
Album Slauson Boy 2 songs has been added to spotify_albums dictionary
Album Mailbox Money songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 10.424344301223755 seconds
Album Crenshaw songs has been added to spotify_albums dictionary
Album Nip Hussle the Great: Vol. 2 songs has been added to spotify_albums dictionary
Album Nip Hussle the Great: Vol. 1 songs has been added to spotify_albums dictionary
Album The Leaks, Vol 1. songs has been added to spotify_albums dictionary
Album Bullets Ain't Got No Name Vol. 2 songs has been added to spotify_albums dictionary
10 playlists completed
Loop #: 10
Elapsed Time: 23.881227731704712 seconds
Album Bullets Ain't Got No Name Vol. 1 songs has been added to spotify_albums dictionary
Album Tmc (Aca

'Oh Wonder'

Album No One Else Can Wear Your Crown (Deluxe) songs has been added to spotify_albums dictionary
Album Ultralife songs has been added to spotify_albums dictionary
Album Oh Wonder songs has been added to spotify_albums dictionary


,album,main_artist_name,release_date,album_popularity,track_number,track_name,disc_number,uri,duration_ms,key,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,track_popularity,explicit
0,Reworked,Snow Patrol,2019-11-08,52,1,Take Back The City (Reworked),1,spotify:track:2E54UgNzMVx4u44TOp4YJD,300173.0,3.0,...,0.555,0.441,0.001980,0.0908,-11.935,0.0360,125.060,0.1620,36,False
1,Reworked,Snow Patrol,2019-11-08,52,2,Open Your Eyes (Reworked),1,spotify:track:6RsmXXyg5r8FqtM338caTl,256560.0,0.0,...,0.577,0.430,0.061100,0.0857,-11.479,0.0360,130.991,0.2240,39,False
2,Reworked,Snow Patrol,2019-11-08,52,3,Time Won't Go Slowly,1,spotify:track:4Ex6THWm2ftd75j9tYBGhz,223333.0,8.0,...,0.407,0.410,0.001240,0.1490,-12.576,0.0903,88.018,0.3530,38,False
3,Reworked,Snow Patrol,2019-11-08,52,4,Chocolate (Reworked),1,spotify:track:0G3ETQWG4sjlh5BOJzRfx8,227160.0,9.0,...,0.511,0.210,0.000273,0.1040,-13.452,0.0286,105.996,0.1640,38,False
4,Reworked,Snow Patrol,2019-11-08,52,5,Set The Fire To The Third Bar (Reworked),1,spotify:track:0xRP7uR86J4q9ZM2g4Fenv,216000.0,7.0,...,0.548,0.532,0.240000,0.1580,-11.782,0.0373,96.087,0.1850,35,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46053,Oh Wonder,Oh Wonder,2015-09-04,53,11,All We Do,1,spotify:track:5GccF2HZmsirzr2fPAlbCK,214174.0,10.0,...,0.596,0.243,0.000299,0.0907,-11.759,0.0471,126.253,0.3740,45,False
46054,Oh Wonder,Oh Wonder,2015-09-04,53,12,Midnight Moon,1,spotify:track:2cXqjJRnV2k6DOk6lzrp4i,210756.0,1.0,...,0.681,0.484,0.006310,0.1150,-9.978,0.0563,143.044,0.6460,25,False
46055,Oh Wonder,Oh Wonder,2015-09-04,53,13,Shark,1,spotify:track:6EqZOuIeYJdF3ybOMnYDnk,187424.0,2.0,...,0.703,0.327,0.292000,0.1120,-7.932,0.0489,149.742,0.1280,30,False
46056,Oh Wonder,Oh Wonder,2015-09-04,53,14,Heart Hope,1,spotify:track:5Zuf9d8Dm2XamT48ynjRHh,249228.0,8.0,...,0.377,0.371,0.050600,0.1080,-13.419,0.0529,122.170,0.0821,27,False


In [16]:
extract_data(top1000artists[700:800], '701','800') #Next 100 artists

'David Bisbal'

Album En Tus Planes songs has been added to spotify_albums dictionary
Album 2En1 songs has been added to spotify_albums dictionary
Album Hijos Del Mar songs has been added to spotify_albums dictionary
Album Tú Y Yo En Vivo songs has been added to spotify_albums dictionary
Album Tú Y Yo (Gold Edition) songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 11.947233438491821 seconds
Album 12 Favoritas songs has been added to spotify_albums dictionary
Album Tú Y Yo songs has been added to spotify_albums dictionary
Album Tú Y Yo (Tour Edition) songs has been added to spotify_albums dictionary
Album Lo Mejor De David Bisbal songs has been added to spotify_albums dictionary
Album Romances songs has been added to spotify_albums dictionary
10 playlists completed
Loop #: 10
Elapsed Time: 22.91700577735901 seconds
Album Live At The Royal Albert Hall songs has been added to spotify_albums dictionary
Album Una Noche en el Teatro Real (Live) songs has been a

'Sting'

Album Remenber Me songs has been added to spotify_albums dictionary
Album My Songs (Live) songs has been added to spotify_albums dictionary
Album My Songs songs has been added to spotify_albums dictionary
Album My Songs (Deluxe) songs has been added to spotify_albums dictionary
Album 44/876 (Deluxe) songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 10.992274284362793 seconds
Album 57TH & 9TH songs has been added to spotify_albums dictionary
Album 57TH & 9TH (Deluxe) songs has been added to spotify_albums dictionary
Album The Last Ship songs has been added to spotify_albums dictionary
Album The Last Ship (Deluxe) songs has been added to spotify_albums dictionary
Album The Last Ship (Super Deluxe) songs has been added to spotify_albums dictionary
10 playlists completed
Loop #: 10
Elapsed Time: 21.917725563049316 seconds
Album 25 Years songs has been added to spotify_albums dictionary
Album Symphonicities songs has been added to spotify_albums

'Beret'

...

'Foals'

Album Collected Reworks Vol. 2 songs has been added to spotify_albums dictionary
Album Collected Reworks Vol. 1 songs has been added to spotify_albums dictionary
Album Part1 Everything Not Saved Will Be Lost (Remixes) songs has been added to spotify_albums dictionary
Album Part 2 Everything Not Saved Will Be Lost songs has been added to spotify_albums dictionary
Album Everything Not Saved Will Be Lost Pt. 1 (Remixes) songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 10.333099603652954 seconds
Album Part 1 Everything Not Saved Will Be Lost songs has been added to spotify_albums dictionary
Album What Went Down songs has been added to spotify_albums dictionary
Album Holy Fire songs has been added to spotify_albums dictionary
Album Foals: Holy Fire Remixes songs has been added to spotify_albums dictionary
Album Total Life Forever songs has been added to spotify_albums dictionary
10 playlists completed
Loop #: 10
Elapsed Time: 19.558870553970337

'Banda Los Recoditos'

Album 30 Aniversario songs has been added to spotify_albums dictionary
Album Lo Más Escuchado De songs has been added to spotify_albums dictionary
Album Perfecta songs has been added to spotify_albums dictionary
Album En Vivo songs has been added to spotify_albums dictionary
Album Los Gustos Que Me Doy songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 13.526591539382935 seconds
Album Las Bandas Románticas songs has been added to spotify_albums dictionary
Album 20 Grandes Con Sabor A Sinaloa songs has been added to spotify_albums dictionary
Album Lola La Bailera songs has been added to spotify_albums dictionary
Album A Bailar De Caballito songs has been added to spotify_albums dictionary
Album Me Está Gustando songs has been added to spotify_albums dictionary
10 playlists completed
Loop #: 10
Elapsed Time: 24.887205123901367 seconds
Album A Ritmo De Banda songs has been added to spotify_albums dictionary
Album Nostalgia songs has been added 

,album,main_artist_name,release_date,album_popularity,track_number,track_name,disc_number,uri,duration_ms,key,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,track_popularity,explicit
0,En Tus Planes,David Bisbal,2020-01-02,70,1,En Tus Planes,1,spotify:track:5fmXWEFh8vMXzKTHN9ezKV,172147.0,11.0,...,0.813,0.780,0.000000,0.0351,-3.768,0.1620,92.014,0.790,62,False
1,En Tus Planes,David Bisbal,2020-01-02,70,2,Si Tú La Quieres,1,spotify:track:2utyR88FRxLvjTuW0US2mk,225413.0,0.0,...,0.757,0.789,0.000000,0.1070,-4.921,0.0835,100.039,0.712,59,False
2,En Tus Planes,David Bisbal,2020-01-02,70,3,Sabrás,1,spotify:track:3ZP8OjNJmf5aSBiZisDlxk,251227.0,2.0,...,0.382,0.712,0.000000,0.0931,-5.482,0.0428,172.004,0.240,46,False
3,En Tus Planes,David Bisbal,2020-01-02,70,4,A Partir De Hoy,1,spotify:track:3Z7JMhKrVBiMvMd0xV1vyI,196803.0,6.0,...,0.652,0.847,0.000000,0.3780,-5.031,0.0552,163.939,0.515,66,False
4,En Tus Planes,David Bisbal,2020-01-02,70,5,Amor Amé,1,spotify:track:6qHdlBWxA5bSPFRV65FYxy,205173.0,3.0,...,0.723,0.919,0.000000,0.1260,-3.907,0.0816,93.065,0.795,52,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29532,Oye Amigo,Banda Los Recoditos,1991,35,6,A Mi Esposa,1,spotify:track:6eIhtY6wIvgLTZByPd3S3I,244846.0,5.0,...,0.730,0.418,0.000013,0.0787,-10.529,0.0456,122.598,0.966,28,False
29533,Oye Amigo,Banda Los Recoditos,1991,35,7,Ritmo De Fiesta,1,spotify:track:6kgao3YtZ7w4QkQO4XYGTz,216842.0,9.0,...,0.743,0.424,0.000197,0.0905,-15.075,0.1240,144.386,0.952,18,False
29534,Oye Amigo,Banda Los Recoditos,1991,35,8,La Loca,1,spotify:track:5T7d4TNZsiSLduv1nyV93s,176222.0,5.0,...,0.618,0.651,0.000002,0.1170,-11.009,0.0647,153.746,0.970,10,False
29535,Oye Amigo,Banda Los Recoditos,1991,35,9,La Cumbia Del Sapito,1,spotify:track:54xqctBDRSUnNRrQY6GvTi,171259.0,7.0,...,0.761,0.588,0.000002,0.1010,-10.927,0.0790,115.214,0.944,8,False


In [18]:
extract_data(top1000artists[800:900], '801','900') #Next 100 artists

'Alina Baraz'

Album It Was Divine songs has been added to spotify_albums dictionary
Album The Color Of You songs has been added to spotify_albums dictionary
Album Urban Flora (Remixes) songs has been added to spotify_albums dictionary
Album Urban Flora songs has been added to spotify_albums dictionary


'C. Tangana'

Album Avida Dollars songs has been added to spotify_albums dictionary
Album Ídolo songs has been added to spotify_albums dictionary


'Nat King Cole'

...

'Nico & Vinz'

Album Black Star Elephant songs has been added to spotify_albums dictionary


'A$AP Mob'

Album Cozy Tapes Vol. 2: Too Cozy songs has been added to spotify_albums dictionary
Album Cozy Tapes: Vol. 1 Friends - songs has been added to spotify_albums dictionary


,album,main_artist_name,release_date,album_popularity,track_number,track_name,disc_number,uri,duration_ms,key,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,track_popularity,explicit
0,It Was Divine,Alina Baraz,2020-04-24,70,1,My Whole Life,1,spotify:track:3WPK4vjLMa9ZxaYetkh47M,162688.0,11.0,...,0.358,0.457,0.000469,0.2590,-11.281,0.0421,180.144,0.336,56,False
1,It Was Divine,Alina Baraz,2020-04-24,70,2,Morocco (feat. 6LACK),1,spotify:track:6cZCrWA68xFTljPmBHPlwJ,141941.0,2.0,...,0.615,0.713,0.001440,0.1540,-3.539,0.0355,78.033,0.514,54,False
2,It Was Divine,Alina Baraz,2020-04-24,70,3,Frank,1,spotify:track:7fUbgLWI7UcYAyjtdKEvkL,214926.0,0.0,...,0.601,0.384,0.045400,0.0845,-9.866,0.0415,129.916,0.172,53,False
3,It Was Divine,Alina Baraz,2020-04-24,70,4,Endlessly,1,spotify:track:7cIPifhS3R4rF4sKi8466d,300574.0,5.0,...,0.687,0.445,0.023700,0.1100,-9.110,0.0635,117.981,0.435,53,False
4,It Was Divine,Alina Baraz,2020-04-24,70,5,Gimme the Wheel (feat. Smino),1,spotify:track:3Slv3vJ64a4bW8yR0llk70,217130.0,11.0,...,0.808,0.579,0.000039,0.1060,-6.538,0.1430,101.951,0.676,52,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61684,Cozy Tapes: Vol. 1 Friends -,A$AP Mob,2016-10-31,65,8,Motivation Foreign (Skit),1,spotify:track:16OdI031GlwjaV6SBPtGtN,125960.0,1.0,...,0.551,0.324,0.000000,0.5800,-20.761,0.7200,107.553,0.657,34,True
61685,Cozy Tapes: Vol. 1 Friends -,A$AP Mob,2016-10-31,65,9,"London Town (feat. A$AP Rocky, A$AP Ant & Play...",1,spotify:track:73ZaCmh5nzvBzbhREnQvcB,162493.0,9.0,...,0.618,0.610,0.000000,0.1080,-7.812,0.3900,126.016,0.529,46,True
61686,Cozy Tapes: Vol. 1 Friends -,A$AP Mob,2016-10-31,65,10,Runner (feat. A$AP Ant & Lil Uzi Vert),1,spotify:track:44gZSuP2nPUvab9ajBrnVM,199987.0,8.0,...,0.856,0.548,0.000000,0.1150,-6.658,0.2050,130.063,0.312,49,True
61687,Cozy Tapes: Vol. 1 Friends -,A$AP Mob,2016-10-31,65,11,"Bachelor (feat. A$AP Rocky, Lil Yachty, Madein...",1,spotify:track:4Nbi6hJWVqM7Ois9G2KKjC,176040.0,1.0,...,0.785,0.607,0.000000,0.1790,-5.103,0.0953,128.077,0.168,50,True


In [19]:
extract_data(top1000artists[900:1000], '901','1000') #Last 100 artists

'MC Kevin o Chris'

Album Evoluiu (ao Vivo) songs has been added to spotify_albums dictionary


'Bonobo'

Album fabric presents Bonobo (DJ Mix) songs has been added to spotify_albums dictionary
Album Migration songs has been added to spotify_albums dictionary
Album The North Borders Tour. — Live. songs has been added to spotify_albums dictionary
Album Late Night Tales: Bonobo songs has been added to spotify_albums dictionary
Album The North Border songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 11.482103824615479 seconds
Album The North Borders songs has been added to spotify_albums dictionary
Album Black Sands Remixed songs has been added to spotify_albums dictionary
Album Black Sands Remixed (Bonus Track Version) songs has been added to spotify_albums dictionary
Album Black Sands songs has been added to spotify_albums dictionary
Album Days To Come songs has been added to spotify_albums dictionary
10 playlists completed
Loop #: 10
Elapsed Time: 22.562177419662476 seconds
Album Recurring (The Live Sessions) songs has been added to spotify_alb

'Two Feet'

...

'Theory of a Deadman'

Album Say Nothing songs has been added to spotify_albums dictionary
Album Wake Up Call songs has been added to spotify_albums dictionary
Album Savages songs has been added to spotify_albums dictionary
Album The Truth Is… (Special Edition) songs has been added to spotify_albums dictionary
Album The Truth Is… (Clean) songs has been added to spotify_albums dictionary
5 playlists completed
Loop #: 5
Elapsed Time: 11.125597715377808 seconds
Album The Truth Is... (Special Edition) songs has been added to spotify_albums dictionary
Album The Truth Is... songs has been added to spotify_albums dictionary
Album Scars & Souvenirs (Special Edition) songs has been added to spotify_albums dictionary
Album Scars & Souvenirs songs has been added to spotify_albums dictionary
Album Gasoline songs has been added to spotify_albums dictionary
10 playlists completed
Loop #: 10
Elapsed Time: 23.077690362930298 seconds
Album Gasoline (Special Edition) songs has been added to spotify_albums dictionary
Album The

'Marian Hill'

Album Unusual songs has been added to spotify_albums dictionary
Album ACT ONE songs has been added to spotify_albums dictionary
Album ACT ONE (The Complete Collection) songs has been added to spotify_albums dictionary
Album Sway songs has been added to spotify_albums dictionary


,album,main_artist_name,release_date,album_popularity,track_number,track_name,disc_number,uri,duration_ms,key,...,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,track_popularity,explicit
0,Evoluiu (ao Vivo),MC Kevin o Chris,2020-02-13,57,1,Venenosa - Ao Vivo,1,spotify:track:4QMzUl7goz1AWynq8DPkmn,172800.0,8.0,...,0.657,0.719,0.000000,0.9530,-8.386,0.606,150.067,0.604,43,True
1,Evoluiu (ao Vivo),MC Kevin o Chris,2020-02-13,57,2,Romance Proibido - Ao Vivo,1,spotify:track:69Si1G3kLuCQLqQF1EbL25,162176.0,11.0,...,0.652,0.798,0.000010,0.3350,-5.167,0.241,150.005,0.557,46,False
2,Evoluiu (ao Vivo),MC Kevin o Chris,2020-02-13,57,3,Senta e Contrai - Ao Vivo,1,spotify:track:3MsfLEkhZvpYqsPXgS9rEN,143829.0,7.0,...,0.725,0.748,0.000005,0.9430,-6.908,0.253,149.739,0.611,32,True
3,Evoluiu (ao Vivo),MC Kevin o Chris,2020-02-13,57,4,Deixa a Pepeca - Ao Vivo,1,spotify:track:7fMCXv3vToSaxagz6Z1oIl,134101.0,4.0,...,0.677,0.802,0.000000,0.3900,-5.070,0.064,112.438,0.760,35,True
4,Evoluiu (ao Vivo),MC Kevin o Chris,2020-02-13,57,5,Babalu - Ao Vivo,1,spotify:track:4p7nG4SSlqsgvul7udrFH8,176363.0,1.0,...,0.724,0.550,0.000000,0.9350,-8.040,0.234,150.005,0.195,45,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66662,Sway,Marian Hill,2015-02-17,58,3,Lips,1,spotify:track:2MHdIP9qDvpHDsJfIXp0j6,198907.0,4.0,...,0.818,0.332,0.000174,0.0986,-11.873,0.269,88.946,0.660,43,False
66663,Sway,Marian Hill,2015-02-17,58,4,Wasted,1,spotify:track:1OIZBoGN33Y1hDdkzo21Hp,217253.0,4.0,...,0.779,0.510,0.468000,0.1330,-8.095,0.362,169.994,0.689,35,False
66664,Sway,Marian Hill,2015-02-17,58,5,Lovit,1,spotify:track:2Ag3LUfgpN6ymEMwDOqKdg,173013.0,5.0,...,0.607,0.384,0.032800,0.1190,-11.934,0.420,36.280,0.274,47,False
66665,Sway,Marian Hill,2015-02-17,58,6,Deep,1,spotify:track:3Yn83dJGBWtL0icQirgyp6,206693.0,6.0,...,0.710,0.256,0.181000,0.0832,-10.112,0.199,75.026,0.285,39,False
